### JSON-очередной формат хранения данных
Практически все результаты наших `GET-` и `POST-` запросов будут возвращаться нам в формате `JSON`. Считывать их можно  методом `.json()`, встроенным в сам requests, однако давайте теперь научимся их ещё и сохранять

---

`JSON`-файлы, логично, хранятся в `.json` файлах! А работать с ними можно с помощью библиотеки... `json`! Начнём с импорта

In [1]:
import json

Для примера работы с JSON файлами заранее был подготовлен файл 'json.json' в папке с кодом. Давайте попробуем его открыть, а заодно и узнаем о новом для нас методе `loads`, или иначе `load-string` методе. Он позволяет получить из строки (иначе содержимого файла) удобный для работы тип данных, будь то `list` или `dict`.

In [2]:
with open('json.json', 'r') as source:
    file_content = source.read()
    
print('file_content :\n')
print(type(file_content), file_content[:300] + '...', sep='\n')

file_content :

<class 'str'>
{"pattern_id":97,"context":{"region":"РњРѕСЃРєРІР°","dealType":"РђСЂРµРЅРґР°","device":"desktop","url":"https://domclick.ru/search/on-map?deal_type=rent&category=living&offer_type=room&ne=56.021286,37.967799%20&sw=55.142226,36.803268&offset=0","page":"/search/on-map","filters":"{'deal_type':'rent','...


Это очень похоже на словарь, однако сейчас это строка. Чтобы это дело исправить нужно лишь понять, что перед нами JSON-файл в естественной среде обитания, а его хорошо парсит и превращает в классический словарь/список метод библиотеки json.

Воспользуемся методом <mark>.loads</mark> для того чтобы работать с возвращаемым значением

In [3]:
data = json.loads(file_content)
print('data :\n')
print(type(data), data, sep='\n')

data :

<class 'dict'>
{'pattern_id': 97, 'context': {'region': 'РњРѕСЃРєРІР°', 'dealType': 'РђСЂРµРЅРґР°', 'device': 'desktop', 'url': 'https://domclick.ru/search/on-map?deal_type=rent&category=living&offer_type=room&ne=56.021286,37.967799%20&sw=55.142226,36.803268&offset=0', 'page': '/search/on-map', 'filters': "{'deal_type':'rent','category':'living','offer_type':['room'],'ne':'56.021286,37.967799%20','sw':'55.142226,36.803268','coordinates':{'sw':'55.142226,36.803268','ne':'56.021286,37.967799%20'}}", 'ab': "{'experiment':'default','features':[],'bucket':13,'params':{'region':''}}"}}


Теперь перед нами никакая не строка, а вполне удобный словарь, параметры которого можно читать и изменять!

Давайте попробуем изменить данные и сохранить их обратно файл. Попутно изучим метод `json.dump`

In [4]:
data['pattern_id'] = False
data['new_field'] = None

with open('json.json', 'w') as file:
    json.dump(data, file)
    
# И тут же проверим изменения
with open('json.json', 'r') as file:
    print(file.read())


{"pattern_id": false, "context": {"region": "\u0420\u045a\u0420\u0455\u0421\u0403\u0420\u0454\u0420\u0406\u0420\u00b0", "dealType": "\u0420\u0452\u0421\u0402\u0420\u00b5\u0420\u0405\u0420\u0491\u0420\u00b0", "device": "desktop", "url": "https://domclick.ru/search/on-map?deal_type=rent&category=living&offer_type=room&ne=56.021286,37.967799%20&sw=55.142226,36.803268&offset=0", "page": "/search/on-map", "filters": "{'deal_type':'rent','category':'living','offer_type':['room'],'ne':'56.021286,37.967799%20','sw':'55.142226,36.803268','coordinates':{'sw':'55.142226,36.803268','ne':'56.021286,37.967799%20'}}", "ab": "{'experiment':'default','features':[],'bucket':13,'params':{'region':''}}"}, "new_field": null}


Как мы видим и поле `pattern_id` изменило своё значение, и появилось поле `new_field`, со своим значением `null`, чего мы и пытались добиться

---

Типы данных, что могут храниться в JSON:
- Список
- Словарь
- Число
- true/false
- Строка
- null

Всё что к этому не относится (как кортеж None'ов в последнем примере), будет сконвертированно в один из возможных типов (в список null'ов), если это возможно

---
### P.s : работа над ошибками
Внешнее сходство дефолтного отображения списка и *JSON*-файла не позволяет вам  влоб сохранять файлы без `json.dump`.

А вот и демонстрация того - почему:

In [5]:
# Сперва определим наши данные
my_data = {
    'bool': True,
    'string': 'hell yeah!',
    'int' : 7,
    'float': 3.1415,
    'list': [False, 'a', 2, 3.0],
    'dict': {'key': 'value'},
    'None': None
}

In [6]:
# Обычная запись словаря в файл

with open('get_error.json', 'w') as gate:
    gate.write(str(my_data))

with open('get_error.json', 'r') as source:
    print(source.read())

{'bool': True, 'string': 'hell yeah!', 'int': 7, 'float': 3.1415, 'list': [False, 'a', 2, 3.0], 'dict': {'key': 'value'}, 'None': None}


In [7]:
# Правильная запись словаря в файл

with open('skip_error.json', 'w') as gate:
    json.dump(my_data,gate)
    
with open('skip_error.json', 'r') as source:
    print(source.read())

{"bool": true, "string": "hell yeah!", "int": 7, "float": 3.1415, "list": [false, "a", 2, 3.0], "dict": {"key": "value"}, "None": null}


Казалось-бы... что такого? Кажется ответы совпадают =/

А теперь давайте попробуем считать первый файл методом `json.loads`

In [8]:
with open('get_error.json', 'r') as source:
    data = json.loads(source.read())

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

И виной тому следующие пункты:

**"Двойные!"** : 
- В *JSON* файлах для разграничения строк используются только двойные кавычки

**Регистр ИмЕеТ значениЕ*** :
- Вместо привычных для нас True/False в файл записываются true/false.

**None != null** : 
- для неопределённых значений в *JSON* используется понятие null, а не None.

---

Теперь зная это мы можем, конечно, научиться записывать в файл без использования библиотеки:

In [9]:
def py_to_json(data):
    content = str(data)
    content = content.replace('True','true')
    content = content.replace('False','false')
    content = content.replace('None','null')
    content = content.replace('\'','"')
    return content

Но и с такой функцие проблемы рано или поздно появятся

In [10]:
data = {
    "who's your daddy?": "Nonessential function",
}

# Ну вы поняли =)

with open('get_error.json', 'w') as gate:
    gate.write(py_to_json(data))

with open('get_error.json', 'r') as source:
    data = json.loads(source.read())

JSONDecodeError: Expecting ':' delimiter: line 1 column 7 (char 6)

Иными словами не стройте велосипед, и пользуйтесь православными `json.loads` & `json.dump`